In [1]:
#Import Library
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  2.0.1     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
# Converts all factors to character class
unfactorize <- function(df){
  for(i in which(sapply(df, class) == "factor")) df[[i]] = as.character(df[[i]])
  return(df)
}

In [3]:
#Load and unfactor data
letters <- unfactorize(read.csv("20201005_AM_FinalMeta.csv"))
#Show data
glimpse(letters)

Observations: 1,222
Variables: 51
$ X                           <int> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13…
$ docsequence                 <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 3…
$ docid                       <chr> "S10003-D023", "S10003-D024", "S10003-D02…
$ docyear                     <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838,…
$ doctype                     <chr> "Letter", "Letter", "Letter", "Letter", "…
$ allsubject                  <chr> "Childbirth; Church attendance; Cities; F…
$ broadsubj                   <chr> "Health; Religion; Communities; Relations…
$ personalevent               <chr> NA, NA, NA, NA, NA, NA, NA, NA, "Physical…
$ wwritten                    <chr> "Baltimore, MD; Maryland; United States; …
$ docauthorid                 <chr> "per0022938", "per0022938", "per0022938",…
$ docauthorname.x             <chr> "Bruns, Jette, 1813-1899", "Bruns, Jette,…
$ docauthorname.y             <chr> "Bruns, Jette, 1813-1899", "Bruns, Jette,…
$ language        

In [4]:
# Drop index variable
letters <- select(letters,-c(1))
names(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname.x"            
[11] "docauthorname.y"             "language"                   
[13] "editor"                      "briefname.x"                
[15] "briefname.y"                 "authrace.x"                 
[17] "authrace.y"                  "nationality.x"              
[19] "nationality.y"               "religion.x"                 
[21] "religion.y"                  "birthyear.x"                
[23] "birthyear.y"                 "deathyear.x"                
[25] "deathyear.y"                 "birthplace.x"               
[27] "birthplace.y"                "deathplace.x"               
[29] "deathplace.y"                "ageatdeath"                 
[31] "authorgender.x"              "authorgender.y"             
[33] "native_occupation.x"         "native_occupation.y"        
[35] "north_american_occupation.x" "north_american_occupation.y"
[37] "year_immigration.x"          "year_immigration.y"         
[39] "cultural_heritage.x"         "cultural_heritage.y"        
[41] "stayed_north_america.x"      "stayed_north_america.y"     
[43] "agewriting"                  "marriagestatus"             
[45] "maternalstatus"              "author_generation"          
[47] "authorgender"                "cultural_heritage"          
[49] "religion"                    "north_american_occupation"

In [5]:
# Cound the number of cases with S9908 in docid
nrow(subset(letters, grepl('S9908',docid)))

[1] 10

Remove these cases. They were identified during sentiment analysis to be much larger files than the other letter files. Upon closer inspection, they appear to be journal entries, or perhaps letters of a prisoner, which is a special case. 

In [6]:
letters <- letters[- grep("S9908", letters$docid),]
nrow(letters)

[1] 1212

## Check variables with both x and y versions.

In [7]:
# Are there differences for docauthorname
letters[which(letters$docauthorname.x != letters$docauthorname.y), c('docauthorname.x', 'docauthorname.y')]

,docauthorname.x,docauthorname.y
216,"Anonymous Russian Jewish Mother, fl. 1910","Anonymous Russian Jewish Mother, 1910"
220,"Anonymous Russian Jewish Male, V. A., fl. 1911","Anonymous Russian Jewish Male, V. A."
947,"Pietersen, Jacobus, fl. 1876","Pietersen, Jacobus, 1876"
981,"Pietersen, Jacobus, fl. 1876","Pietersen, Jacobus, 1876"


In [8]:
# Keeping docauthorname.x because more nuanced.
# Drop docauthorname.y.
letters <- select(letters,-c(docauthorname.y))

# Change variable name
names(letters)[names(letters)=="docauthorname.x"] <- "docauthorname"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname.x"                 "briefname.y"                
[15] "authrace.x"                  "authrace.y"                 
[17] "nationality.x"               "nationality.y"              
[19] "religion.x"                  "religion.y"                 
[21] "birthyear.x"                 "birthyear.y"                
[23] "deathyear.x"                 "deathyear.y"                
[25] "birthplace.x"                "birthplace.y"               
[27] "deathplace.x"                "deathplace.y"               
[29] "ageatdeath"                  "authorgender.x"             
[31] "authorgender.y"              "native_occupation.x"        
[33] "native_occupation.y"         "north_american_occupation.x"
[35] "north_american_occupation.y" "year_immigration.x"         
[37] "year_immigration.y"          "cultural_heritage.x"        
[39] "cultural_heritage.y"         "stayed_north_america.x"     
[41] "stayed_north_america.y"      "agewriting"                 
[43] "marriagestatus"              "maternalstatus"             
[45] "author_generation"           "authorgender"               
[47] "cultural_heritage"           "religion"                   
[49] "north_american_occupation"

[1] 1212

In [9]:
# Are there differences for briefname.x
letters[which(letters$briefname.x != letters$briefname.y), c('briefname.x', 'briefname.y')]

,briefname.x,briefname.y
216,Anonymous Russian Jewish Mother,"Anonymous Russian Jewish Mother, 1910"
947,Jacobus Pietersen,"Jacobus Pietersen, 1876"
981,Jacobus Pietersen,"Jacobus Pietersen, 1876"


In [10]:
sample(letters$briefname.x, 10)

[1] "Susannah Moodie"                               
 [2] "Sarah Stretch Harris"                          
 [3] "William Bell"                                  
 [4] "Iwao Matsushita"                               
 [5] "Jette Bruns"                                   
 [6] "Anonymous Norwegian Immigrant in Galveston, TX"
 [7] "Robert Harris"                                 
 [8] "Sarah Stretch Harris"                          
 [9] "Critchlow Harris"                              
[10] "Sarah Stretch Harris"

In [11]:
# Keeping briefname.x because the other values for that variable do not include date.
letters <- select(letters,-c(briefname.y))

# Change variable name
names(letters)[names(letters)=="briefname.x"] <- "briefname"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "authrace.x"                 
[15] "authrace.y"                  "nationality.x"              
[17] "nationality.y"               "religion.x"                 
[19] "religion.y"                  "birthyear.x"                
[21] "birthyear.y"                 "deathyear.x"                
[23] "deathyear.y"                 "birthplace.x"               
[25] "birthplace.y"                "deathplace.x"               
[27] "deathplace.y"                "ageatdeath"                 
[29] "authorgender.x"              "authorgender.y"             
[31] "native_occupation.x"         "native_occupation.y"        
[33] "north_american_occupation.x" "north_american_occupation.y"
[35] "year_immigration.x"          "year_immigration.y"         
[37] "cultural_heritage.x"         "cultural_heritage.y"        
[39] "stayed_north_america.x"      "stayed_north_america.y"     
[41] "agewriting"                  "marriagestatus"             
[43] "maternalstatus"              "author_generation"          
[45] "authorgender"                "cultural_heritage"          
[47] "religion"                    "north_american_occupation"

[1] 1212

In [12]:
# Check to race category counts
summary(as.factor(letters$authrace.x))
summary(as.factor(letters$authrace.y))

Asian Black White 
  148     1  1063

Asian Black White 
  148     1  1063

In [13]:
# Confirm that the Asian letters have one author
summary(as.factor(letters$docauthorname[letters$authrace.x=="Asian"]))

Matsushita, Iwao, 1892-1972 
                        148

Race data is not very informative because there is only one Black authored letter and all the Asian authored letters are by the same person.  

In [14]:
# Drop both versions of the race variable.
letters <- select(letters,-c(authrace.x, authrace.y))

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality.x"              
[15] "nationality.y"               "religion.x"                 
[17] "religion.y"                  "birthyear.x"                
[19] "birthyear.y"                 "deathyear.x"                
[21] "deathyear.y"                 "birthplace.x"               
[23] "birthplace.y"                "deathplace.x"               
[25] "deathplace.y"                "ageatdeath"                 
[27] "authorgender.x"              "authorgender.y"             
[29] "native_occupation.x"         "native_occupation.y"        
[31] "north_american_occupation.x" "north_american_occupation.y"
[33] "year_immigration.x"          "year_immigration.y"         
[35] "cultural_heritage.x"         "cultural_heritage.y"        
[37] "stayed_north_america.x"      "stayed_north_america.y"     
[39] "agewriting"                  "marriagestatus"             
[41] "maternalstatus"              "author_generation"          
[43] "authorgender"                "cultural_heritage"          
[45] "religion"                    "north_american_occupation"

[1] 1212

In [15]:
# Are there differences for nationality
sum(is.na(letters$nationality.x))
sum(is.na(letters$nationality.y))

[1] 421

[1] 422

In [16]:
# Omit y version of the nationality variable because more NAs
letters <- select(letters,-c(nationality.y))

# Change variable name
names(letters)[names(letters)=="nationality.x"] <- "nationality"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "religion.x"                  "religion.y"                 
[17] "birthyear.x"                 "birthyear.y"                
[19] "deathyear.x"                 "deathyear.y"                
[21] "birthplace.x"                "birthplace.y"               
[23] "deathplace.x"                "deathplace.y"               
[25] "ageatdeath"                  "authorgender.x"             
[27] "authorgender.y"              "native_occupation.x"        
[29] "native_occupation.y"         "north_american_occupation.x"
[31] "north_american_occupation.y" "year_immigration.x"         
[33] "year_immigration.y"          "cultural_heritage.x"        
[35] "cultural_heritage.y"         "stayed_north_america.x"     
[37] "stayed_north_america.y"      "agewriting"                 
[39] "marriagestatus"              "maternalstatus"             
[41] "author_generation"           "authorgender"               
[43] "cultural_heritage"           "religion"                   
[45] "north_american_occupation"

[1] 1212

In [17]:
# Making sure the record for Hedy Lamar has been corrected or filtered.
nrow(letters$briefname[which(letters$docauthorid == "per0021755")])

NULL

In [18]:
# Omit x and y versions of the religion variable because a composite one is already in df.
letters <- select(letters,-c(religion.x, religion.y))

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear.x"                 "birthyear.y"                
[17] "deathyear.x"                 "deathyear.y"                
[19] "birthplace.x"                "birthplace.y"               
[21] "deathplace.x"                "deathplace.y"               
[23] "ageatdeath"                  "authorgender.x"             
[25] "authorgender.y"              "native_occupation.x"        
[27] "native_occupation.y"         "north_american_occupation.x"
[29] "north_american_occupation.y" "year_immigration.x"         
[31] "year_immigration.y"          "cultural_heritage.x"        
[33] "cultural_heritage.y"         "stayed_north_america.x"     
[35] "stayed_north_america.y"      "agewriting"                 
[37] "marriagestatus"              "maternalstatus"             
[39] "author_generation"           "authorgender"               
[41] "cultural_heritage"           "religion"                   
[43] "north_american_occupation"

[1] 1212

In [19]:
# Are there differences for birthyear
letters[which(letters$birthyear.x != letters$birthyear.y), c('birthyear.x', 'birthyear.y')]

birthyear.x,birthyear.y


In [20]:
# Omit y version of birthyear
letters <- select(letters,-c(birthyear.y))

# Change variable name
names(letters)[names(letters)=="birthyear.x"] <- "birthyear"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear.x"                
[17] "deathyear.y"                 "birthplace.x"               
[19] "birthplace.y"                "deathplace.x"               
[21] "deathplace.y"                "ageatdeath"                 
[23] "authorgender.x"              "authorgender.y"             
[25] "native_occupation.x"         "native_occupation.y"        
[27] "north_american_occupation.x" "north_american_occupation.y"
[29] "year_immigration.x"          "year_immigration.y"         
[31] "cultural_heritage.x"         "cultural_heritage.y"        
[33] "stayed_north_america.x"      "stayed_north_america.y"     
[35] "agewriting"                  "marriagestatus"             
[37] "maternalstatus"              "author_generation"          
[39] "authorgender"                "cultural_heritage"          
[41] "religion"                    "north_american_occupation"

[1] 1212

In [21]:
# Omit y version of birthyear
letters <- select(letters,-c(deathyear.y))

# Change variable name
names(letters)[names(letters)=="deathyear.x"] <- "deathyear"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear"                  
[17] "birthplace.x"                "birthplace.y"               
[19] "deathplace.x"                "deathplace.y"               
[21] "ageatdeath"                  "authorgender.x"             
[23] "authorgender.y"              "native_occupation.x"        
[25] "native_occupation.y"         "north_american_occupation.x"
[27] "north_american_occupation.y" "year_immigration.x"         
[29] "year_immigration.y"          "cultural_heritage.x"        
[31] "cultural_heritage.y"         "stayed_north_america.x"     
[33] "stayed_north_america.y"      "agewriting"                 
[35] "marriagestatus"              "maternalstatus"             
[37] "author_generation"           "authorgender"               
[39] "cultural_heritage"           "religion"                   
[41] "north_american_occupation"

[1] 1212

In [22]:
# Are there diferences for birthplace?
sum(is.na(letters$birthplace.x))
sum(is.na(letters$birthplace.y))

[1] 136

[1] 137

In [23]:
# Omit y version of birthplace
letters <- select(letters,-c(birthplace.y))

# Change variable name
names(letters)[names(letters)=="birthplace.x"] <- "birthplace"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear"                  
[17] "birthplace"                  "deathplace.x"               
[19] "deathplace.y"                "ageatdeath"                 
[21] "authorgender.x"              "authorgender.y"             
[23] "native_occupation.x"         "native_occupation.y"        
[25] "north_american_occupation.x" "north_american_occupation.y"
[27] "year_immigration.x"          "year_immigration.y"         
[29] "cultural_heritage.x"         "cultural_heritage.y"        
[31] "stayed_north_america.x"      "stayed_north_america.y"     
[33] "agewriting"                  "marriagestatus"             
[35] "maternalstatus"              "author_generation"          
[37] "authorgender"                "cultural_heritage"          
[39] "religion"                    "north_american_occupation"

[1] 1212

In [24]:
# Are there diferences for deathplace?
sum(is.na(letters$deathplace.x))
sum(is.na(letters$deathplace.y))

[1] 677

[1] 677

In [25]:
# Omit y version of deathplace
letters <- select(letters,-c(deathplace.y))

# Change variable name
names(letters)[names(letters)=="deathplace.x"] <- "deathplace"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear"                  
[17] "birthplace"                  "deathplace"                 
[19] "ageatdeath"                  "authorgender.x"             
[21] "authorgender.y"              "native_occupation.x"        
[23] "native_occupation.y"         "north_american_occupation.x"
[25] "north_american_occupation.y" "year_immigration.x"         
[27] "year_immigration.y"          "cultural_heritage.x"        
[29] "cultural_heritage.y"         "stayed_north_america.x"     
[31] "stayed_north_america.y"      "agewriting"                 
[33] "marriagestatus"              "maternalstatus"             
[35] "author_generation"           "authorgender"               
[37] "cultural_heritage"           "religion"                   
[39] "north_american_occupation"

[1] 1212

In [26]:
# Drop x and y versions of authorgender variable 
letters <- select(letters,-c(authorgender.x, authorgender.y))

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear"                  
[17] "birthplace"                  "deathplace"                 
[19] "ageatdeath"                  "native_occupation.x"        
[21] "native_occupation.y"         "north_american_occupation.x"
[23] "north_american_occupation.y" "year_immigration.x"         
[25] "year_immigration.y"          "cultural_heritage.x"        
[27] "cultural_heritage.y"         "stayed_north_america.x"     
[29] "stayed_north_america.y"      "agewriting"                 
[31] "marriagestatus"              "maternalstatus"             
[33] "author_generation"           "authorgender"               
[35] "cultural_heritage"           "religion"                   
[37] "north_american_occupation"

[1] 1212

In [27]:
sum(is.na(letters$native_occupation.x))
sum(is.na(letters$native_occupation.y))

[1] 406

[1] 507

In [28]:
# Omit y version of native_occupation
letters <- select(letters,-c(native_occupation.y))

# Change variable name
names(letters)[names(letters)=="native_occupation.x"] <- "native_occupation"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"                 "docid"                      
 [3] "docyear"                     "doctype"                    
 [5] "allsubject"                  "broadsubj"                  
 [7] "personalevent"               "wwritten"                   
 [9] "docauthorid"                 "docauthorname"              
[11] "language"                    "editor"                     
[13] "briefname"                   "nationality"                
[15] "birthyear"                   "deathyear"                  
[17] "birthplace"                  "deathplace"                 
[19] "ageatdeath"                  "native_occupation"          
[21] "north_american_occupation.x" "north_american_occupation.y"
[23] "year_immigration.x"          "year_immigration.y"         
[25] "cultural_heritage.x"         "cultural_heritage.y"        
[27] "stayed_north_america.x"      "stayed_north_america.y"     
[29] "agewriting"                  "marriagestatus"             
[31] "maternalstatus"              "author_generation"          
[33] "authorgender"                "cultural_heritage"          
[35] "religion"                    "north_american_occupation"

[1] 1212

In [29]:
# Drop x and y versions of north_american_occupation because a composite variables already exists.
letters <- select(letters,-c(north_american_occupation.x, north_american_occupation.y))

# Check df
names(letters)
nrow(letters)

[1] "docsequence"               "docid"                    
 [3] "docyear"                   "doctype"                  
 [5] "allsubject"                "broadsubj"                
 [7] "personalevent"             "wwritten"                 
 [9] "docauthorid"               "docauthorname"            
[11] "language"                  "editor"                   
[13] "briefname"                 "nationality"              
[15] "birthyear"                 "deathyear"                
[17] "birthplace"                "deathplace"               
[19] "ageatdeath"                "native_occupation"        
[21] "year_immigration.x"        "year_immigration.y"       
[23] "cultural_heritage.x"       "cultural_heritage.y"      
[25] "stayed_north_america.x"    "stayed_north_america.y"   
[27] "agewriting"                "marriagestatus"           
[29] "maternalstatus"            "author_generation"        
[31] "authorgender"              "cultural_heritage"        
[33] "religion"                  "north_american_occupation"

[1] 1212

In [30]:
# Are there differences for year_immigration
letters[which(letters$year_immigration.x != letters$year_immigration.y), c('year_immigration.x', 'year_immigration.y')]

year_immigration.x,year_immigration.y


In [31]:
# Omit y version of year_immigration
letters <- select(letters,-c(year_immigration.y))

# Change variable name
names(letters)[names(letters)=="year_immigration.x"] <- "year_immigration"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"               "docid"                    
 [3] "docyear"                   "doctype"                  
 [5] "allsubject"                "broadsubj"                
 [7] "personalevent"             "wwritten"                 
 [9] "docauthorid"               "docauthorname"            
[11] "language"                  "editor"                   
[13] "briefname"                 "nationality"              
[15] "birthyear"                 "deathyear"                
[17] "birthplace"                "deathplace"               
[19] "ageatdeath"                "native_occupation"        
[21] "year_immigration"          "cultural_heritage.x"      
[23] "cultural_heritage.y"       "stayed_north_america.x"   
[25] "stayed_north_america.y"    "agewriting"               
[27] "marriagestatus"            "maternalstatus"           
[29] "author_generation"         "authorgender"             
[31] "cultural_heritage"         "religion"                 
[33] "north_american_occupation"

[1] 1212

In [32]:
# Are there differences for year_immigration
letters[which(letters$cultural_heritage.x != letters$cultural_heritage), c('cultural_heritage.x', 'cultural_heritage')]

cultural_heritage.x,cultural_heritage


In [33]:
# Drop x and y versions of cultural_heritage because a composite variables already exists.
letters <- select(letters,-c(cultural_heritage.x, cultural_heritage.y))

# Check df
names(letters)
nrow(letters)

[1] "docsequence"               "docid"                    
 [3] "docyear"                   "doctype"                  
 [5] "allsubject"                "broadsubj"                
 [7] "personalevent"             "wwritten"                 
 [9] "docauthorid"               "docauthorname"            
[11] "language"                  "editor"                   
[13] "briefname"                 "nationality"              
[15] "birthyear"                 "deathyear"                
[17] "birthplace"                "deathplace"               
[19] "ageatdeath"                "native_occupation"        
[21] "year_immigration"          "stayed_north_america.x"   
[23] "stayed_north_america.y"    "agewriting"               
[25] "marriagestatus"            "maternalstatus"           
[27] "author_generation"         "authorgender"             
[29] "cultural_heritage"         "religion"                 
[31] "north_american_occupation"

[1] 1212

In [34]:
# Are there differences for year_immigration
letters[which(letters$stayed_north_america.x != letters$stayed_north_america.y), c('stayed_north_america.x', 'stayed_north_america.y')]

,stayed_north_america.x,stayed_north_america.y
1,Stayed,No
2,Stayed,No
3,Stayed,No
4,Stayed,No
5,Stayed,No
6,Stayed,No
7,Stayed,No
8,Stayed,No
9,Stayed,No
10,Stayed,No


In [35]:
# Drop y version of stayed_north_america because of errors (too many nos).
letters <- select(letters,-c(stayed_north_america.y))

# Change variable name
names(letters)[names(letters)=="stayed_north_america.x"] <- "stayed_north_america"

# Check df
names(letters)
nrow(letters)

[1] "docsequence"               "docid"                    
 [3] "docyear"                   "doctype"                  
 [5] "allsubject"                "broadsubj"                
 [7] "personalevent"             "wwritten"                 
 [9] "docauthorid"               "docauthorname"            
[11] "language"                  "editor"                   
[13] "briefname"                 "nationality"              
[15] "birthyear"                 "deathyear"                
[17] "birthplace"                "deathplace"               
[19] "ageatdeath"                "native_occupation"        
[21] "year_immigration"          "stayed_north_america"     
[23] "agewriting"                "marriagestatus"           
[25] "maternalstatus"            "author_generation"        
[27] "authorgender"              "cultural_heritage"        
[29] "religion"                  "north_american_occupation"

[1] 1212

In [36]:
# Converts character to factor class
factorize <- function(df){
  for(i in which(sapply(df, class) == "character")) df[[i]] = as.factor(df[[i]])
  return(df)
}

In [37]:
# Turn characters into factors and get summary
letters  <- factorize(letters)
summary(letters)

  docsequence             docid         docyear       doctype    
 Min.   :  2.00   S10003-D023:   1   Min.   :1804   Letter:1212  
 1st Qu.: 25.00   S10003-D024:   1   1st Qu.:1858                
 Median : 55.00   S10003-D025:   1   Median :1869                
 Mean   : 72.78   S10003-D026:   1   Mean   :1882                
 3rd Qu.:110.25   S10003-D027:   1   3rd Qu.:1909                
 Max.   :239.00   S10003-D028:   1   Max.   :1979                
                  (Other)    :1206   NA's   :20                  
                                                         allsubject  
 Correspondence; Intellectual life                            :  10  
 Internment camps; Politics                                   :   8  
 Winter; Environment                                          :   6  
 Correspondence; Internment camps; Intellectual life; Politics:   5  
 Photographs; Intellectual life                               :   5  
 Business; Sons; Economics; Relationships           

In [38]:
# Remove underscores from variable names
names(letters)<-gsub("\\_","",names(letters))
names(letters)

[1] "docsequence"             "docid"                  
 [3] "docyear"                 "doctype"                
 [5] "allsubject"              "broadsubj"              
 [7] "personalevent"           "wwritten"               
 [9] "docauthorid"             "docauthorname"          
[11] "language"                "editor"                 
[13] "briefname"               "nationality"            
[15] "birthyear"               "deathyear"              
[17] "birthplace"              "deathplace"             
[19] "ageatdeath"              "nativeoccupation"       
[21] "yearimmigration"         "stayednorthamerica"     
[23] "agewriting"              "marriagestatus"         
[25] "maternalstatus"          "authorgeneration"       
[27] "authorgender"            "culturalheritage"       
[29] "religion"                "northamericanoccupation"

In [39]:
glimpse(letters)

Observations: 1,212
Variables: 30
$ docsequence             <int> 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 3…
$ docid                   <fct> S10003-D023, S10003-D024, S10003-D025, S10003…
$ docyear                 <int> 1836, 1836, 1837, 1837, 1838, 1838, 1838, 183…
$ doctype                 <fct> Letter, Letter, Letter, Letter, Letter, Lette…
$ allsubject              <fct> "Childbirth; Church attendance; Cities; Farms…
$ broadsubj               <fct> Health; Religion; Communities; Relationships;…
$ personalevent           <fct> NA, NA, NA, NA, NA, NA, NA, NA, Physical illn…
$ wwritten                <fct> "Baltimore, MD; Maryland; United States; Mid-…
$ docauthorid             <fct> per0022938, per0022938, per0022938, per002293…
$ docauthorname           <fct> "Bruns, Jette, 1813-1899", "Bruns, Jette, 181…
$ language                <fct> English; German, English; German, English; Ge…
$ editor                  <fct> "Schroeder, Adoplh E., tr.; Geisberg, Carla S…
$ briefname       

In [40]:
write.csv(letters, "20201119_AM_Meta2Merge.csv")